In [80]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [81]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    model_name="llama-3.1-8b-instant"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [82]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-ii-backend-itc/job/R-68103")
page_data = loader.load().pop().page_content
print(page_data)





















Software Engineer II - Backend, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous

In [84]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [86]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)

{'Software Engineer II - Backend, ITC': {'role': 'Software Engineer II - Backend, ITC', 'experience': {'min': 2, 'max': 4, 'years': 'years of software development experience'}, 'skills': ['Experience working in a highly collaborative, multi-discipline development team environment', 'Experience working in a distributed/cloud-based environment (e.g. AWS Services, GCE, Rackspace, Azure etc.)', 'Experience in micro service architecture, domain driven design, and RESTful Services using Java', 'Experience working with Java and Spring Boot framework mandatorily', 'Experience in Test Driven development – with Junit, Mockito', 'Experience with monitoring and tracing tools like Splunk, SignalFx, New Relic etc.', 'Experience working with NoSQL data stores like Cassandra, DynamoDB, MongoDB etc.', 'Experience working with relational data stores like MySQL, Oracle etc.', 'Experience with continuous integration, unit testing, static analysis, and automated integration tests. Continuous delivery exper

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("my_portfolio.csv")
df.head(2)

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio


In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [90]:
job = next(iter(json_res.values()))
skills = job["skills"]

skills

['Experience working in a highly collaborative, multi-discipline development team environment',
 'Experience working in a distributed/cloud-based environment (e.g. AWS Services, GCE, Rackspace, Azure etc.)',
 'Experience in micro service architecture, domain driven design, and RESTful Services using Java',
 'Experience working with Java and Spring Boot framework mandatorily',
 'Experience in Test Driven development – with Junit, Mockito',
 'Experience with monitoring and tracing tools like Splunk, SignalFx, New Relic etc.',
 'Experience working with NoSQL data stores like Cassandra, DynamoDB, MongoDB etc.',
 'Experience working with relational data stores like MySQL, Oracle etc.',
 'Experience with continuous integration, unit testing, static analysis, and automated integration tests. Continuous delivery experience preferred',
 'Working knowledge of Scrum and agile principles',
 'Comfortable working in a fast-paced, results-oriented environment',
 'Comfortable working with globally dis

In [91]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'http

In [93]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are mrX, a business development executive at CompX. CompX is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of CompX 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase CompX's portfolio: {link_list}
        Remember you are mrX, BDE at CompX. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Your Digital Transformation with CompX

Dear Hiring Manager,

I came across the job description for a Software Engineer II - Backend at Nike, and I was impressed by the exciting opportunity to contribute to the Supply Chain and Planning Technology team. As a Business Development Executive at CompX, an AI & Software Consulting company, I'd like to introduce you to our capabilities in fulfilling your needs.

At CompX, we specialize in facilitating seamless integration of business processes through automated tools, empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise aligns perfectly with the requirements mentioned in the job description.

Some of our notable projects include:

- **DevOps Mastery**: We've helped numerous clients achieve DevOps excellence through our expertise in tools like Jenkins, Docker, and Kubernetes. You can explore our DevOps portfolio here: http